In [2]:
from manim import *

TITLE_SIZE = 40
SUBTITLE_SIZE = 24

config.verbosity = "WARNING"

In [4]:
%%manim -qh Isoperimetry


class Isoperimetry(Scene):
    def construct(self):
        title = Text("Isoperimetry Ratio", font_size=TITLE_SIZE, color=BLUE).to_edge(UP + LEFT, buff=0.5)

        self.play(Create(title))
        self.wait(3)

        vertices = [1, 2, 3, 4, 5, 6, 7]
        edges = [(1, 2), (2, 3), (3, 4), (4, 5), (6, 1), (1, 4), (2, 5), (3, 6), (4, 6), (1, 5), (2, 7), (5, 7)]

        g = Graph(vertices, edges, layout="circular", layout_scale=2.5, labels=True)

        self.play(Write(g))
        self.wait(5)

        subset_vertices = [1, 4, 5]
        boundary_edges = [(1, 2), (3, 4), (6, 1), (2, 5), (4, 6), (5, 7)]

        # Color the subset vertices
        for vertex in subset_vertices:
            g.vertices[vertex].set_color(RED)

        self.play(*[FadeToColor(g.vertices[vertex], RED) for vertex in subset_vertices])
        self.wait(2)

        for edge in boundary_edges:
            g.edges[edge].set_color(GREEN)

        self.play(*[FadeToColor(g.edges[edge], GREEN) for edge in boundary_edges])
        self.wait(2)

        self.play(g.animate.scale(0.7).to_edge(LEFT, buff=1.5))

        text = MathTex(r"\theta(S) = \frac{|\text{boundary edges}|}{|\text{S}|}", font_size=40)
        text2 = MathTex(r" = \frac{6}{3} = 2", font_size=40)

        text.move_to(RIGHT * 2)
        self.play(Write(text))
        self.wait(3)
        self.play(text.animate.shift(LEFT * 1.0))
        text2.next_to(text, RIGHT, buff=0.5)
        self.play(Write(text2))
        self.wait(3)

        self.play(FadeOut(g), FadeOut(text), FadeOut(text2), FadeOut(title))


Manim Community v0.18.1

[10/11/24 10:47:54] INFO     Animation 0 : Partial movie file written in                   ]8;id=542283;file:///Users/geo/anaconda3/lib/python3.11/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=834241;file:///Users/geo/anaconda3/lib/python3.11/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/geo/Documents/courses/CS_2952Q/spectral-graph-theory/                         
                             chapter20/media/videos/chapter20/1080p60/partial_movie_files/                         
                             Isoperimetry/3529212410_1058300966_223132457.mp4'                                     

[10/11/24 10:47:55] INFO     Animation 1 : Partial movie file written in                   ]8;id=720360;file:///Users/geo/anaconda3/lib/python3.11/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=214597;file:///Users/geo/anaconda3/lib/python3.11/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/geo/Documents/courses/CS_2952Q/spectral-graph-theory/                         
                             chapter20/media/videos/chapter20/1080p60/partial_movie_files/                         
                             Isoperimetry/3040924799_4071659840_3825828318.mp4'                                    

                    INFO     Writing 1 to media/Tex/6ecf9f51170c1a70.tex                    ]8;id=528195;file:///Users/geo/anaconda3/lib/python3.11/site-packages/manim/utils/tex_file_writing.py\tex_file_writing.py]8;;\:]8;id=421346;file:///Users/geo/anaconda3/lib/python3.11/site-packages/manim/utils/tex_file_writing.py#109\109]8;;\

[10/11/24 10:47:56] INFO     Writing 2 to media/Tex/2b7ffb3c38a5a6e0.tex                    ]8;id=76566;file:///Users/geo/anaconda3/lib/python3.11/site-packages/manim/utils/tex_file_writing.py\tex_file_writing.py]8;;\:]8;id=958662;file:///Users/geo/anaconda3/lib/python3.11/site-packages/manim/utils/tex_file_writing.py#109\109]8;;\

...

[10/11/24 10:48:15] INFO     Animation 13 : Partial movie file written in                  ]8;id=731048;file:///Users/geo/anaconda3/lib/python3.11/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=285057;file:///Users/geo/anaconda3/lib/python3.11/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/geo/Documents/courses/CS_2952Q/spectral-graph-theory/                         
                             chapter20/media/videos/chapter20/1080p60/partial_movie_files/                         
                             Isoperimetry/3040924799_2475379470_2426121071.mp4'                                    

[10/11/24 10:48:17] INFO     Animation 14 : Partial movie file written in                  ]8;id=115334;file:///Users/geo/anaconda3/lib/python3.11/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=210235;file:///Users/geo/anaconda3/lib/python3.11/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/geo/Documents/courses/CS_2952Q/spectral-graph-theory/                         
                             chapter20/media/videos/chapter20/1080p60/partial_movie_files/                         
                             Isoperimetry/3040924799_1352151339_1425331443.mp4'                                    

                    INFO     Combining to Movie file.                                      ]8;id=221753;file:///Users/geo/anaconda3/lib/python3.11/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=75367;file:///Users/geo/anaconda3/lib/python3.11/site-packages/manim/scene/scene_file_writer.py#617\617]8;;\

                    INFO                                                                   ]8;id=237435;file:///Users/geo/anaconda3/lib/python3.11/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=499006;file:///Users/geo/anaconda3/lib/python3.11/site-packages/manim/scene/scene_file_writer.py#737\737]8;;\
                             File ready at                                                                         
                             '/Users/geo/Documents/courses/CS_2952Q/spectral-graph-theory/                         
                             chapter20/media/videos/chapter20/1080p60/Isoperimetry.mp4'                            
                                                                                                                   

                    INFO     Rendered Isoperimetry                                                     ]8;id=327916;file:///Users/geo/anaconda3/lib/python3.11/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=107858;file:///Users/geo/anaconda3/lib/python3.11/site-packages/manim/scene/scene.py#247\247]8;;\
                             Played 15 animations                                                                  